# Import the dataset

In [1]:
from io import StringIO
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import pandas as pd
def zip2df(link):
    # Open the url link
    url = urllib.request.urlopen(link)
    
    # Open as zip file
    with ZipFile(BytesIO(url.read())) as my_zip_file:
        for contained_file in my_zip_file.namelist():
            fzip=my_zip_file.open(contained_file)
            print(contained_file)
            data=fzip.read()
    # Convert bytes object to string object
    s=str(data,'utf-8')
    data = StringIO(s) 
    # convert it to pandas DataFrame as normal csv file
    df=pd.read_csv(data)
    return df

In [2]:
link_data='https://github.com/duonghung86/Fatality-crashes/raw/master/Codes/Mapped%20combined%20data.zip'
df=zip2df(link_data)
df.head()

Mapped combined data.csv


,Crash_ID,Unit_Nbr,Prsn_Nbr,Prsn_Type_ID,Prsn_Occpnt_Pos_ID,Prsn_Injry_Sev_ID,Prsn_Age,Prsn_Ethnicity_ID,Prsn_Gndr_ID,Prsn_Ejct_ID,...,Road_Constr_Zone_Wrkr_Fl,At_Intrsct_Fl,Wthr_Cond_ID,Light_Cond_ID,Road_Algn_ID,Surf_Cond_ID,Traffic_Cntl_ID,Latitude,Longitude,Unit_Desc_ID
0,15657177,1,1,DRIVER,FRONT LEFT,UNKNOWN,16.0,HISPANIC,MALE,NO,...,N,N,CLEAR,"DARK, LIGHTED","CURVE, LEVEL",DRY,NONE,32.280231,-97.746616,MOTOR VEHICLE
1,16473665,1,1,DRIVER,FRONT LEFT,NOT INJURED,48.0,WHITE,MALE,NO,...,N,Y,CLEAR,DAYLIGHT,"STRAIGHT, HILLCREST",DRY,SIGNAL LIGHT,30.660685,-93.893906,MOTOR VEHICLE
2,16473665,2,1,DRIVER,FRONT LEFT,NON-INCAPACITATING INJURY,58.0,WHITE,FEMALE,NO,...,N,Y,CLEAR,DAYLIGHT,"STRAIGHT, HILLCREST",DRY,SIGNAL LIGHT,30.660685,-93.893906,MOTOR VEHICLE
3,16871051,1,1,DRIVER,FRONT LEFT,NOT INJURED,68.0,WHITE,MALE,NO,...,N,N,CLEAR,DAYLIGHT,"STRAIGHT, LEVEL",DRY,MARKED LANES,33.203920,-96.596654,MOTOR VEHICLE
4,16871051,3,1,DRIVER,FRONT LEFT,NOT INJURED,67.0,WHITE,FEMALE,NO,...,N,N,CLEAR,DAYLIGHT,"STRAIGHT, LEVEL",DRY,MARKED LANES,33.203920,-96.596654,MOTOR VEHICLE


In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df.drop(columns=['Crash_ID','Prsn_Nbr'],inplace=True)

In [5]:
df.Prsn_Injry_Sev_ID.unique()

array(['UNKNOWN', 'NOT INJURED', 'NON-INCAPACITATING INJURY',
       'INCAPACITATING INJURY', 'POSSIBLE INJURY', 'KILLED'], dtype=object)

In [6]:
df=df[df.Prsn_Injry_Sev_ID!='UNKNOWN'].copy()
df.shape

(488849, 29)

## Check feature types

In [7]:
feat_types=df.dtypes

In [8]:
feat_types.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

# Numerical variables

In [9]:
# name of numerical variables
num_var=feat_types[feat_types!='object']
num_var=num_var.index
num_var

Index(['Unit_Nbr', 'Prsn_Age', 'Crash_Speed_Limit', 'Latitude', 'Longitude'], dtype='object')

In [10]:
# Person age
df[num_var].describe().round(2)

,Unit_Nbr,Prsn_Age,Crash_Speed_Limit,Latitude,Longitude
count,488849.00,488849.00,488849.00,488849.00,488849.00
mean,1.59,38.99,43.67,30.79,-97.39
std,0.69,16.39,16.00,1.89,2.49
min,1.00,0.00,-1.00,25.86,-106.63
25%,1.00,25.00,35.00,29.66,-98.20
50%,1.00,36.00,45.00,30.23,-96.96
75%,2.00,51.00,55.00,32.68,-95.51
max,12.00,119.00,85.00,36.50,-93.54


In [14]:
# export GPS coordinates to plot 
df[['Latitude', 'Longitude','Prsn_Injry_Sev_ID']].to_csv('GPS vs severity.csv',index=False)

![map](https://github.com/duonghung86/Fatality-crashes/raw/master/Figures/Injury%20severity%20in%20Texas.png)

# Categorical variables

In [39]:
cate_vars=df.dtypes[df.dtypes=='object']
cate_vars=cate_vars.index
cate_vars

Index(['Prsn_Type_ID', 'Prsn_Occpnt_Pos_ID', 'Prsn_Injry_Sev_ID',
       'Prsn_Ethnicity_ID', 'Prsn_Gndr_ID', 'Prsn_Ejct_ID', 'Prsn_Rest_ID',
       'Prsn_Airbag_ID', 'Prsn_Helmet_ID', 'Day_of_Week', 'Rpt_Rdwy_Sys_ID',
       'Rpt_Road_Part_ID', 'Toll_Road_Fl', 'Road_Constr_Zone_Fl',
       'Road_Constr_Zone_Wrkr_Fl', 'At_Intrsct_Fl', 'Wthr_Cond_ID',
       'Light_Cond_ID', 'Road_Algn_ID', 'Surf_Cond_ID', 'Traffic_Cntl_ID',
       'Unit_Desc_ID'],
      dtype='object')

## Time

In [17]:
time=df['Crash_Date']+' '+df['Crash_Time']
time=pd.to_datetime(time)

In [27]:
df.drop(columns=['Crash_Date','Crash_Time'],inplace=True)

In [32]:
df['Crash_month']=time.apply(lambda x: x.month)
df['Crash_day']=time.apply(lambda x: x.day)
df['Crash_hour']=time.apply(lambda x: x.hour)
df['Crash_min']=time.apply(lambda x: x.minute)

In [33]:
df.head()

,Unit_Nbr,Prsn_Type_ID,Prsn_Occpnt_Pos_ID,Prsn_Injry_Sev_ID,Prsn_Age,Prsn_Ethnicity_ID,Prsn_Gndr_ID,Prsn_Ejct_ID,Prsn_Rest_ID,Prsn_Airbag_ID,...,Road_Algn_ID,Surf_Cond_ID,Traffic_Cntl_ID,Latitude,Longitude,Unit_Desc_ID,Crash_month,Crash_day,Crash_hour,Crash_min
1,1,DRIVER,FRONT LEFT,NOT INJURED,48.0,WHITE,MALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, HILLCREST",DRY,SIGNAL LIGHT,30.660685,-93.893906,MOTOR VEHICLE,6,15,11,0
2,2,DRIVER,FRONT LEFT,NON-INCAPACITATING INJURY,58.0,WHITE,FEMALE,NO,SHOULDER & LAP BELT,"DEPLOYED, FRONT",...,"STRAIGHT, HILLCREST",DRY,SIGNAL LIGHT,30.660685,-93.893906,MOTOR VEHICLE,6,15,11,0
3,1,DRIVER,FRONT LEFT,NOT INJURED,68.0,WHITE,MALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, LEVEL",DRY,MARKED LANES,33.203920,-96.596654,MOTOR VEHICLE,6,12,9,53
4,3,DRIVER,FRONT LEFT,NOT INJURED,67.0,WHITE,FEMALE,NO,SHOULDER & LAP BELT,"DEPLOYED, SIDE",...,"STRAIGHT, LEVEL",DRY,MARKED LANES,33.203920,-96.596654,MOTOR VEHICLE,6,12,9,53
5,1,DRIVER,FRONT LEFT,NOT INJURED,36.0,HISPANIC,FEMALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, GRADE",DRY,WARNING SIGN,29.792394,-95.746539,MOTOR VEHICLE,6,4,15,7


In [65]:
min(time)

Timestamp('2019-06-01 00:00:00')

In [66]:
max(time)

Timestamp('2019-11-30 23:57:00')

In [37]:
df['Day_of_Week'].value_counts()/len(df)*100

FRI    17.165423
TUE    15.341752
WED    15.285497
THU    15.193649
MON    14.537823
SAT    12.836684
SUN     9.639173
Name: Day_of_Week, dtype: float64

## Simple categorical variable (2 unique values)

In [41]:
for feat in cate_vars:
    if len(df[feat].unique())==2:
        print(feat)
        print(df[feat].value_counts()/len(df)*100)

Toll_Road_Fl
N    98.247721
Y     1.752279
Name: Toll_Road_Fl, dtype: float64
Road_Constr_Zone_Fl
N    95.025049
Y     4.974951
Name: Road_Constr_Zone_Fl, dtype: float64
Road_Constr_Zone_Wrkr_Fl
N    98.304384
Y     1.695616
Name: Road_Constr_Zone_Wrkr_Fl, dtype: float64
At_Intrsct_Fl
N    65.587124
Y    34.412876
Name: At_Intrsct_Fl, dtype: float64


In [42]:
for feat in cate_vars:
    if len(df[feat].unique())==2:
        print(feat)
        df[feat]=np.where(df[feat]==df[feat].unique()[0],0,1)
df.head()

Toll_Road_Fl
Road_Constr_Zone_Fl
Road_Constr_Zone_Wrkr_Fl
At_Intrsct_Fl


,Unit_Nbr,Prsn_Type_ID,Prsn_Occpnt_Pos_ID,Prsn_Injry_Sev_ID,Prsn_Age,Prsn_Ethnicity_ID,Prsn_Gndr_ID,Prsn_Ejct_ID,Prsn_Rest_ID,Prsn_Airbag_ID,...,Road_Algn_ID,Surf_Cond_ID,Traffic_Cntl_ID,Latitude,Longitude,Unit_Desc_ID,Crash_month,Crash_day,Crash_hour,Crash_min
1,1,DRIVER,FRONT LEFT,NOT INJURED,48.0,WHITE,MALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, HILLCREST",DRY,SIGNAL LIGHT,30.660685,-93.893906,MOTOR VEHICLE,6,15,11,0
2,2,DRIVER,FRONT LEFT,NON-INCAPACITATING INJURY,58.0,WHITE,FEMALE,NO,SHOULDER & LAP BELT,"DEPLOYED, FRONT",...,"STRAIGHT, HILLCREST",DRY,SIGNAL LIGHT,30.660685,-93.893906,MOTOR VEHICLE,6,15,11,0
3,1,DRIVER,FRONT LEFT,NOT INJURED,68.0,WHITE,MALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, LEVEL",DRY,MARKED LANES,33.203920,-96.596654,MOTOR VEHICLE,6,12,9,53
4,3,DRIVER,FRONT LEFT,NOT INJURED,67.0,WHITE,FEMALE,NO,SHOULDER & LAP BELT,"DEPLOYED, SIDE",...,"STRAIGHT, LEVEL",DRY,MARKED LANES,33.203920,-96.596654,MOTOR VEHICLE,6,12,9,53
5,1,DRIVER,FRONT LEFT,NOT INJURED,36.0,HISPANIC,FEMALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, GRADE",DRY,WARNING SIGN,29.792394,-95.746539,MOTOR VEHICLE,6,4,15,7


## Other categorical variables

### Target - Severity

In [45]:
df['Prsn_Injry_Sev_ID'].unique()

array(['NOT INJURED', 'NON-INCAPACITATING INJURY',
       'INCAPACITATING INJURY', 'POSSIBLE INJURY', 'KILLED'], dtype=object)

In [46]:
sever_map={'NOT INJURED':0, 'NON-INCAPACITATING INJURY':2,
       'INCAPACITATING INJURY':3, 'POSSIBLE INJURY':1, 'KILLED':4}

In [52]:
df_final=df['Prsn_Injry_Sev_ID'].map(sever_map).copy()
df_final.columns=['Injury Severity']
df_final.head()

1    0
2    2
3    0
4    0
5    0
Name: Prsn_Injry_Sev_ID, dtype: int64

In [54]:
df_final.shape

(488849,)

In [55]:
#drop the injury severity
df.drop(columns=['Prsn_Injry_Sev_ID'],inplace=True)

In [59]:
df_final=pd.merge(df_final,df,left_index=True, right_index=True)

In [60]:
df_final

,Prsn_Injry_Sev_ID,Unit_Nbr,Prsn_Type_ID,Prsn_Occpnt_Pos_ID,Prsn_Age,Prsn_Ethnicity_ID,Prsn_Gndr_ID,Prsn_Ejct_ID,Prsn_Rest_ID,Prsn_Airbag_ID,...,Road_Algn_ID,Surf_Cond_ID,Traffic_Cntl_ID,Latitude,Longitude,Unit_Desc_ID,Crash_month,Crash_day,Crash_hour,Crash_min
1,0,1,DRIVER,FRONT LEFT,48.0,WHITE,MALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, HILLCREST",DRY,SIGNAL LIGHT,30.660685,-93.893906,MOTOR VEHICLE,6,15,11,0
2,2,2,DRIVER,FRONT LEFT,58.0,WHITE,FEMALE,NO,SHOULDER & LAP BELT,"DEPLOYED, FRONT",...,"STRAIGHT, HILLCREST",DRY,SIGNAL LIGHT,30.660685,-93.893906,MOTOR VEHICLE,6,15,11,0
3,0,1,DRIVER,FRONT LEFT,68.0,WHITE,MALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, LEVEL",DRY,MARKED LANES,33.203920,-96.596654,MOTOR VEHICLE,6,12,9,53
4,0,3,DRIVER,FRONT LEFT,67.0,WHITE,FEMALE,NO,SHOULDER & LAP BELT,"DEPLOYED, SIDE",...,"STRAIGHT, LEVEL",DRY,MARKED LANES,33.203920,-96.596654,MOTOR VEHICLE,6,12,9,53
5,0,1,DRIVER,FRONT LEFT,36.0,HISPANIC,FEMALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, GRADE",DRY,WARNING SIGN,29.792394,-95.746539,MOTOR VEHICLE,6,4,15,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495424,0,1,DRIVER,FRONT LEFT,68.0,WHITE,MALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, LEVEL",UNKNOWN,NONE,28.954878,-97.987513,MOTOR VEHICLE,11,29,11,13
495425,0,1,DRIVER,FRONT LEFT,44.0,WHITE,MALE,NO,SHOULDER & LAP BELT,"DEPLOYED, FRONT",...,"STRAIGHT, LEVEL",DRY,STOP SIGN,32.756880,-94.354907,MOTOR VEHICLE,11,25,11,6
495426,1,2,DRIVER,FRONT LEFT,57.0,BLACK,FEMALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, LEVEL",DRY,STOP SIGN,32.756880,-94.354907,MOTOR VEHICLE,11,25,11,6
495427,0,1,DRIVER,FRONT LEFT,16.0,WHITE,FEMALE,NO,SHOULDER & LAP BELT,NOT DEPLOYED,...,"STRAIGHT, LEVEL",DRY,STOP SIGN,31.279209,-94.579816,MOTOR VEHICLE,9,3,16,15


In [64]:
print(df_final['Prsn_Injry_Sev_ID'].value_counts()/N*100)

0    81.454191
1    11.454048
2     5.569204
3     1.216940
4     0.305616
Name: Prsn_Injry_Sev_ID, dtype: float64


### Others

In [63]:
cate_vars=df_final.dtypes[df_final.dtypes=='object']
cate_vars=cate_vars.index
N=len(df_final)
for feat in cate_vars:
    if len(df_final[feat].unique())>2:
        print(feat, len(df_final[feat].unique()))
        print(df_final[feat].value_counts()/N*100)


Prsn_Type_ID 8
DRIVER                                           97.942514
DRIVER OF MOTORCYCLE TYPE VEHICLE                 0.803316
PEDESTRIAN                                        0.616141
PEDALCYCLIST                                      0.282909
PASSENGER/OCCUPANT                                0.251202
UNKNOWN                                           0.063414
OTHER (EXPLAIN IN NARRATIVE)                      0.039276
PASSENGER/OCCUPANT ON MOTORCYCLE TYPE VEHICLE     0.001227
Name: Prsn_Type_ID, dtype: float64
Prsn_Occpnt_Pos_ID 14
FRONT LEFT                                           98.863657
PEDESTRIAN, PEDALCYCLIST, OR MOTORIZED CONVEYANCE     0.918893
FRONT RIGHT                                           0.076506
OUTSIDE VEHICLE                                       0.045822
FRONT CENTER                                          0.033139
UNKNOWN                                               0.032321
OTHER (EXPLAIN IN NARRATIVE)                          0.017388
OTHER IN VEHICL

In [ ]:
# convert to dummies

In [68]:
for feat in cate_vars:
    num_uni=len(df_final[feat].unique())
    if num_uni>2:
        print(feat)
        dummies=pd.get_dummies(df_final[feat],prefix=feat).iloc[:,:num_uni-1]
        #print(dummies)
        df_final.drop(columns=[feat],inplace=True)
        df_final=df_final.join(dummies)
df_final

Prsn_Type_ID
Prsn_Occpnt_Pos_ID
Prsn_Ethnicity_ID
Prsn_Gndr_ID
Prsn_Ejct_ID
Prsn_Rest_ID
Prsn_Airbag_ID
Prsn_Helmet_ID
Day_of_Week
Rpt_Rdwy_Sys_ID
Rpt_Road_Part_ID
Wthr_Cond_ID
Light_Cond_ID
Road_Algn_ID
Surf_Cond_ID
Traffic_Cntl_ID
Unit_Desc_ID


,Prsn_Injry_Sev_ID,Unit_Nbr,Prsn_Age,Toll_Road_Fl,Crash_Speed_Limit,Road_Constr_Zone_Fl,Road_Constr_Zone_Wrkr_Fl,At_Intrsct_Fl,Latitude,Longitude,...,Traffic_Cntl_ID_SIGNAL LIGHT WITH RED LIGHT RUNNING CAMERA,Traffic_Cntl_ID_STOP SIGN,Traffic_Cntl_ID_WARNING SIGN,Unit_Desc_ID_MOTOR VEHICLE,Unit_Desc_ID_MOTORIZED CONVEYANCE,Unit_Desc_ID_NON-CONTACT,Unit_Desc_ID_OTHER (EXPLAIN IN NARRATIVE),Unit_Desc_ID_PEDALCYCLIST,Unit_Desc_ID_PEDESTRIAN,Unit_Desc_ID_TOWED/PUSHED/TRAILER
1,0,1,48.0,0,35,0,0,0,30.660685,-93.893906,...,0,0,0,1,0,0,0,0,0,0
2,2,2,58.0,0,35,0,0,0,30.660685,-93.893906,...,0,0,0,1,0,0,0,0,0,0
3,0,1,68.0,0,45,0,0,1,33.203920,-96.596654,...,0,0,0,1,0,0,0,0,0,0
4,0,3,67.0,0,45,0,0,1,33.203920,-96.596654,...,0,0,0,1,0,0,0,0,0,0
5,0,1,36.0,0,35,0,0,1,29.792394,-95.746539,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495424,0,1,68.0,0,70,0,0,1,28.954878,-97.987513,...,0,0,0,1,0,0,0,0,0,0
495425,0,1,44.0,0,55,0,0,1,32.756880,-94.354907,...,0,1,0,1,0,0,0,0,0,0
495426,1,2,57.0,0,55,0,0,1,32.756880,-94.354907,...,0,1,0,1,0,0,0,0,0,0
495427,0,1,16.0,0,40,0,0,0,31.279209,-94.579816,...,0,1,0,1,0,0,0,0,0,0


In [69]:
compression_opts = dict(method='zip',
                        archive_name='final data.csv')  
df_final.to_csv('final data.zip', index=False,
          compression=compression_opts) 